In [1]:
import numpy as np
from pathlib import Path

In [2]:
with Path("../16.in").open() as f:
    data = f.read().strip().splitlines()
    data = np.array([list(line) for line in data])

with Path("../16_test.in").open() as f:
    testdata = f.read().strip().splitlines()
    testdata = np.array([list(line) for line in testdata])


In [3]:
def get_evaluated_directions(data, ray):
    # Check the current tile
    y, x, dy, dx = ray
    tile = data[y, x]

    if tile == ".":
        # Continue in the same direction
        return [(dy, dx)]

    elif tile == "|":
        if dy != 0:
            # Continue in the same direction
            return [(dy, dx)]
        else:
            # Change direction
            return [(1, 0), (-1, 0)]

    elif tile == "-":
        if dx != 0:
            # Continue in the same direction
            return [(dy, dx)]
        else:
            # Change direction
            return [(0, 1), (0, -1)]

    elif tile == "/":
        if dx != 0:
            # Change direction
            return [(-dx, 0)]
        else:
            # Change direction
            return [(0, -dy)]

    elif tile == "\\":
        if dx != 0:
            # Change direction
            return [(dx, 0)]
        else:
            # Change direction
            return [(0, dy)]


In [4]:
def get_visited(data, rays):
    visited = {*rays}

    while rays:
        y, x, dy, dx = rays.pop()

        # Move in that direction
        x += dx
        y += dy

        new_ray = (y, x, dy, dx)

        if not (0 <= x < data.shape[1] and 0 <= y < data.shape[0]):
            # Out of bounds
            continue
        elif new_ray in visited:
            # Already visited
            continue

        visited.add(new_ray)

        # Check the current tile
        evaluated_directions = get_evaluated_directions(data, new_ray)
        for dy, dx in evaluated_directions:
            rays.append((y, x, dy, dx))

    return visited


In [8]:
def get_starting_ray(data):
    y, x, dy, dx = 0, 0, 0, 1
    directions = get_evaluated_directions(data, (y, x, dy, dx))
    return (y, x, *directions[0])


In [9]:
def solve1(data):
    ray = get_starting_ray(data)
    visited = get_visited(data, [ray])
    num_energized = len(set([ray[:2] for ray in visited]))
    return num_energized

In [10]:
assert solve1(testdata) == 46

In [11]:
solve1(data)

7074

## Part II

In [12]:
def get_initial_rays(data):
    starting_rays = []

    # Initialize rays from the edge points
    for y in range(data.shape[0]):
        starting_rays.append((y, 0, 0, 1))
        starting_rays.append((y, data.shape[1] -1, 0, -1))
    for x in range(data.shape[1]):
        starting_rays.append((0, x, 1, 0))
        starting_rays.append((data.shape[0] - 1, x, -1, 0))

    # Update each ray direction(s) according to the tile it is on
    for i, ray in enumerate(starting_rays):
        new_dirs = get_evaluated_directions(data, ray)
        if len(new_dirs) > 1:
            # Split the ray into two
            starting_rays[i] = (ray[0], ray[1], *new_dirs[0])
            starting_rays.append((ray[0], ray[1], *new_dirs[1]))
        else:
            # Update the direction
            starting_rays[i] = (ray[0], ray[1], *new_dirs[0])

    return starting_rays


In [13]:
def solve2(data):
    initial_rays = get_initial_rays(data)
    max_energized = 0
    max_ray = None

    for ray in initial_rays:
        visited = get_visited(data, [ray])
        num_energized = len(set([ray[:2] for ray in visited]))
        if num_energized > max_energized:
            max_energized = num_energized
            max_ray = ray

    print(f"{max_ray} -> {max_energized}")
    return max_energized

assert solve2(testdata) == 51

(0, 3, 1, 0) -> 51


In [14]:
solve2(data)

(0, 9, 1, 0) -> 7530


7530